In [1]:
# importing required libraries
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_csv('/content/drive/MyDrive/balanced_data.csv')

In [4]:
data.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,35929,2100,443,2744,49916,6,14653,65,3,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,1,195043,2210,52673,2608,53,17,160,91765,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,2,58449,1337,45245,1593,53,17,27,295,2,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,3,70012,696,58208,865,53,17,80,53034,4,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,4,31918,2777,443,1590,50506,6,119,71,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


## NIDS implementation using DL algorithms for NSL-KDD dataset

In [5]:
! pip install scikit-plot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding,SimpleRNN, GRU,Activation,Flatten
from tensorflow.keras.layers import Conv1D,MaxPool1D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,CSVLogger
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence
from sklearn.metrics import (precision_score, recall_score,confusion_matrix,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
import pandas as pd
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import matplotlib.pyplot as plt
import scikitplot as skplt
#import scikitplot as skplt

In [7]:
import tensorflow.keras
print('keras: %s' % tensorflow.keras.__version__)

keras: 2.11.0


## Convert into test and train data

In [8]:
data.drop(data.columns[0] , inplace=True , axis=1)

In [9]:
data.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,35929,2100,443,2744,49916,6,14653,65,3,1,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,195043,2210,52673,2608,53,17,160,91765,2,2,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,58449,1337,45245,1593,53,17,27,295,2,2,...,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,70012,696,58208,865,53,17,80,53034,4,2,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,31918,2777,443,1590,50506,6,119,71,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [10]:
data.describe()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,...,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06,1.851553e+06
mean,7.828045e+04,1.764915e+03,4.185722e+04,2.435506e+03,7.427576e+03,9.361445e+00,2.496818e+03,1.660826e+07,8.434010e+00,9.136713e+00,...,4.700876e+00,-3.262906e+03,8.493366e+04,3.873955e+04,1.494644e+05,6.249542e+04,1.074609e+07,6.402986e+05,1.123092e+07,1.025749e+07
std,6.547856e+04,1.032903e+03,2.125624e+04,1.509093e+03,1.749527e+04,5.072592e+00,5.924219e+03,3.515372e+07,6.728427e+02,8.939532e+02,...,5.737102e+02,1.207486e+06,6.604447e+05,3.973162e+05,1.017588e+06,5.886217e+05,2.722876e+07,5.290920e+06,2.802688e+07,2.697066e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.200000e+01,1.000000e+00,0.000000e+00,...,0.000000e+00,-5.368707e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.566100e+04,1.333000e+03,3.441600e+04,1.409000e+03,5.300000e+01,6.000000e+00,5.100000e+01,1.170000e+02,1.000000e+00,1.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.272000e+04,1.543000e+03,5.065400e+04,2.558000e+03,8.000000e+01,6.000000e+00,1.170000e+02,3.182300e+04,2.000000e+00,2.000000e+00,...,1.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.217810e+05,2.205000e+03,5.794600e+04,2.610000e+03,4.430000e+02,1.700000e+01,3.860000e+02,5.166715e+06,5.000000e+00,5.000000e+00,...,2.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.492120e+05,8.237000e+03,6.553500e+04,9.697000e+03,6.553500e+04,1.700000e+01,2.599500e+04,1.200000e+08,2.197590e+05,2.919220e+05,...,2.135570e+05,1.260000e+02,1.070000e+08,7.420000e+07,1.070000e+08,1.070000e+08,1.200000e+08,7.660000e+07,1.200000e+08,1.200000e+08


In [11]:
data.columns

Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
  

# **Data Normalization**

In [12]:
# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [13]:
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns

In [14]:
# using standard scaler for normalizing
std_scaler = StandardScaler()
def normalization(df,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
  return df

In [15]:
# calling the normalization() function
data = normalization(data.copy(),numeric_col)

In [16]:
# data after normalization
data.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,-0.646799,0.324411,-1.948333,0.204423,2.428566,-0.662668,2.051947,-0.472445,-0.008076,-0.009102,...,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN
1,1.783219,0.430907,0.508829,0.114303,-0.421518,1.505849,-0.394452,-0.469837,-0.009562,-0.007983,...,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN
2,-0.302869,-0.414284,0.159378,-0.558287,-0.421518,1.505849,-0.416902,-0.472439,-0.009562,-0.007983,...,0.002735,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN
3,-0.126277,-1.034865,0.769223,-1.040696,-0.421518,1.505849,-0.407956,-0.470938,-0.006590,-0.007983,...,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN
4,-0.708056,0.979845,-1.948333,-0.560275,2.462290,-0.662668,-0.401372,-0.472445,-0.011049,-0.009102,...,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.39466,-0.121018,-0.40072,-0.38032,BENIGN


# **One-hot-encoding**

In [17]:
# selecting numeric attributes columns from data
non_numeric_col = data.select_dtypes(exclude='number').columns

In [18]:
non_numeric_col

Index(['Label'], dtype='object')

In [19]:
# creating a dataframe with only categorical attributes
categorical = data[non_numeric_col]
categorical.head()

,Label
0,BENIGN
1,BENIGN
2,BENIGN
3,BENIGN
4,BENIGN


In [20]:
# one-hot-encoding categorical attributes using pandas.get_dummies() function
categorical = pd.get_dummies(categorical,columns=non_numeric_col)
categorical.head()

,Label_BENIGN,Label_Bot,Label_DDoS,Label_DoS GoldenEye,Label_DoS Hulk,Label_DoS Slowhttptest,Label_DoS slowloris,Label_FTP-Patator,Label_Heartbleed,Label_Infiltration,Label_PortScan,Label_SSH-Patator
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0


# **Concatenate with original data :-)**

In [21]:
final_concatenated = pd.concat([ data ,categorical] , axis=1)

In [22]:
final_concatenated.drop(['Label'],axis=1 , inplace=True)

In [23]:
final_concatenated.to_csv('/content/drive/MyDrive/clean_data.csv')